<a href="https://colab.research.google.com/github/svetaU/DGP/blob/main/two_layer_deep_gp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

An implementation of deep gaussian process with two layers using hamiltonian monte carlo sampling of the full posterior. Inspired by papers and online articles: https://doi.org/10.48550/arxiv.2012.08015 ; https://colindcarroll.com/2019/04/11/hamiltonian-monte-carlo-from-scratch/ ; https://bayesianbrad.github.io/posts/2019_hmc.html

# Libraries

In [26]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
import scipy.stats as st
import math
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation, writers
print(tf.__version__)

2.8.2


# Mount colab drives for data

In [2]:
import sys, subprocess
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  print("You are in Google Colab environment!")
  from google.colab import drive
  drive.mount('/content/drive')
else:
  print("This notebook is best run in Google Colab.")

You are in Google Colab environment!
Mounted at /content/drive


# HMC

In [3]:
def leapfrog(q, p, dUdq, num_steps, step_size, random_step=False,m=None):
  if random_step:
    step_size = np.random.uniform(step_size*0.8,step_size*1.2,1)
  if m is None:
    m = np.ones(len(p))
  q, p = np.copy(q), np.copy(p)
  p -= step_size * dUdq(q) / 2.0  
  leap_path = [np.copy(q)]
  for _ in range(num_steps - 1):
      q += step_size * p/m  
      p -= step_size * dUdq(q) 
      leap_path.append(np.copy(q))
  q += step_size * p/m  
  p -= step_size * dUdq(q) / 2.0
  leap_path.append(np.copy(q)) 
  return (q,-p,leap_path)


In [4]:
def hamiltonian_monte_carlo(U, grad_U, current_position, steps=1, delta_t=0.5, change_step = False, masses = None):
  q = np.copy(current_position)
  p = np.random.normal(0.0,1.0,len(q)) 
  current_momentum = np.copy(p)
  if masses is None:
    masses = np.ones(len(p))
  q, p, path = leapfrog(q,p,grad_U,steps,delta_t,change_step, masses)
  current_U = U(current_position)
  current_K = sum(current_momentum*current_momentum/masses) / 2
  proposed_U = U(q)
  proposed_K = sum(p*p/masses) / 2
  if (np.log(np.random.rand()) < (current_U-proposed_U+current_K-proposed_K)):
    return (q,proposed_U,1,path) # accept
  else:
    return (current_position, current_U,0,None) # reject

# Posterior for Gaussian Process (GP)
The model ($Y_n$ is observed GP, $W_n$ is hidden layer GP of the same dim) 
$$ 
Y_n | W ∼ \mathcal{N}_n(0,\tau^2(K_{\theta_y}(W) +g\mathbb{I}_n)),\;\; W_n \sim \mathcal{N}(0,K_{\theta_w}(X_n)),\;\; K_{\theta}^{ij} = e^{-\frac{||x_i-x_j||^2}{\theta}}
$$

With thus defined probabilities, the log likelihoods are (|A| is determinate of a matrix A)

$$
log \mathcal{L}(Y_n|W,\theta_y,g) \propto - \frac{n}{2}log(n\hat{\tau}^2)-\frac{1}{2}log|K_{\theta_y}(W)+g\mathbb{I}_n|
$$
 
$$
log \mathcal{L}(W|X,\theta_w) \propto -\frac{1}{2}log|K_{\theta_w}(X)| - \frac{1}{2} W^\top K^{-1}_{\theta_w}(X)W
$$

with $\hat{\tau}^2$ is defined as a point estimate (not sampled by MCMC)
$$
\hat{\tau}^2=\frac{Y^\top(K_{\theta_y}(W) + g\mathbb{I}_n)^{-1}Y}{n}
$$
via MLE with reference prior ($1/\tau^2$).

Priors for other hyperparameters are gamma distributions.

Altogether, up to a constant, the posterior for the model params looks like
$$
-log(\pi(W,\vec{\theta},g|D_n)) = -log\mathcal{L}(Y_n|W,\theta_y,g) - log \mathcal{L}(W|X_n,\theta_w) - (a_{\theta_y}-1)log(\theta_y)+b_{\theta_y}\theta_y - (a_{\theta_w}-1)log(\theta_w)+b_{\theta_w}\theta_w - (a_{g}-1)log(g)+b_{g}g
$$

with $D_n$ being the data $(Y_n,X_n)$. Here $a, b$ enter gamma pdf as $p(x;a,b) = \frac{x^{a-1}e^{-bx}b^a}{\Gamma(a)}$

In [46]:
class TLdeepGP():

  def __init__(self, **kwargs):
    self.data = ([],[])
    self.a_ty =1.1
    self.b_ty = 1.1
    self.a_tw =1.1
    self.b_tw = 1.1
    self.a_g =1.1
    self.b_g = 1.1

  def set_data(self,data):
    self.data = np.copy(data)
    self.x = self.data[0]
    self.y = self.data[1]
    self.y = self.y - np.mean(self.y)
    self.n = len(self.y)

  def log_likelihood(self,params): # -log(p)
  # We should have total (3 + number of points) params that go like (g,theta_w,theta_y,W)
    noise_matrix = params[0]*tf.eye(num_rows = self.n,dtype=tf.float32)
    det_noise = tf.linalg.det(noise_matrix)
    inv_noise = tfp.math.lu_matrix_inverse(*tf.linalg.lu(noise_matrix))
    x = tf.convert_to_tensor(params[3:], dtype=tf.float32)
    x_row = tf.expand_dims(x, axis=1)
    x_col = tf.expand_dims(x, axis=0)
    dist = tf.maximum(0.,tf.pow(x_row,2) + tf.pow(x_col,2) - 2.*tf.matmul(x_row,x_col))
    print(x)
    print(dist)
    t = 0
    return t   

  def tf_log_likelihood(self,params): # -log(p)
    y = 0
    return y 

  def log_priors(self,params): # -log(\pi)
    p_params = - (self.a_g - 1.0)*math.log(params[0]) + self.b_g*params[0] \
    - (self.a_tw - 1.0)*math.log(params[1]) + self.b_tw*params[1] \
    - (self.a_ty - 1.0)*math.log(params[2]) + self.b_yt*params[2] 
    return p_params

  def log_likelihood_grad(self,params): 
    #params = tf.convert_to_tensor(params, dtype=tf.float32)
    #with tf.GradientTape() as tape:
    #  tape.watch(params)
    #  y = self.tf_log_likelihood(params)
    #return tape.gradient(y,params).numpy()
    return 0

  def log_priors_grad(self, params):
    return 0

  def log_prob(self, params):
    return self.log_likelihood(params) + self.log_priors(params)

  def log_prob_grad(self,params):
    return self.log_likelihood_grad(params) + self.log_priors_grad(params)


# Fit parameters

In [42]:
def fit_dgp_w_hmc():
  target_mean = [2.,3.]
  target_cov = [[2.5,0.4],[0.4,1.6]] 
  sigma1 = round(math.sqrt(target_cov[0][0]),2)
  sigma2 = round(math.sqrt(target_cov[1][1]),2)
  rho = round(target_cov[0][1]/sigma1/sigma2,2)
  sample_size = 5
  data_sample_x, data_sample_y = np.random.multivariate_normal(target_mean,target_cov,size=sample_size).T
  model_fit = TLdeepGP()
  model_fit.set_data((data_sample_x,data_sample_y))
  param_init = [0.1,1.,2.,1.,2.,3.,4.,5.]
  model_fit.log_likelihood(param_init)

In [47]:
sim_dots = fit_dgp_w_hmc()

tf.Tensor([1. 2. 3. 4. 5.], shape=(5,), dtype=float32)
tf.Tensor(
[[ 0.  1.  4.  9. 16.]
 [ 1.  0.  1.  4.  9.]
 [ 4.  1.  0.  1.  4.]
 [ 9.  4.  1.  0.  1.]
 [16.  9.  4.  1.  0.]], shape=(5, 5), dtype=float32)


# Infer data